In [1]:
# Import Dependencies
import pandas as pd

In [2]:
# Create a path to the csv and read it into a Pandas DataFrame
csv_path = "../Data/home_prices.csv"
rent_path = "../Data/yearly_rent.csv"
home_df = pd.read_csv(csv_path, encoding='utf8', engine='python')
rent_df = pd.read_csv(rent_path, encoding='utf8', engine='python')

In [3]:
rent_df.head()

,Neighborhood,Rent_2011,Rent_2012,Rent_2013,Rent_2014,Rent_2015,Rent_2016,Rent_2017,City,Avg Rent,Lat,Lng
0,Bayview,30723,28821,30433,35338,42870,45681,45747,San Francisco,37087.57,37.730416,-122.384424
1,Bernal Heights,34471,35739,38924,43654,53977,54833,53741,San Francisco,45048.43,37.738950,-122.415201
2,Buena Vista,42407,45678,49364,53889,61646,65690,61917,San Francisco,54370.14,37.806504,-122.420778
3,Corona Heights,41051,44269,48263,52768,61781,64072,59849,San Francisco,53150.43,37.761812,-122.443185
4,Cow Hollow,52856,52816,56455,62256,75947,78557,71952,San Francisco,64405.57,37.798033,-122.439646


In [4]:
# Changing from columns to rows (convert dataframe from wide to long)
melted_df = pd.melt(rent_df,id_vars=["Neighborhood"],
                    value_vars=[i for i in list(rent_df.columns.values[1:8])],
                    var_name = "Year", value_name = "Annual Rent")

# Parse "Year" values to reflect year
melted_df["Year"] = melted_df["Year"].str.replace("Rent_", "")
melted_df["Year"] = melted_df["Year"].astype(int)

In [5]:
melted_df.head()

,Neighborhood,Year,Annual Rent
0,Bayview,2011,30723
1,Bernal Heights,2011,34471
2,Buena Vista,2011,42407
3,Corona Heights,2011,41051
4,Cow Hollow,2011,52856


In [6]:
#merging two data frames
rent_df2 = pd.merge(melted_df,rent_df.loc[:,["Neighborhood","City","Lat","Lng"]])
rent_df2.head()

,Neighborhood,Year,Annual Rent,City,Lat,Lng
0,Bayview,2011,30723,San Francisco,37.730416,-122.384424
1,Bayview,2012,28821,San Francisco,37.730416,-122.384424
2,Bayview,2013,30433,San Francisco,37.730416,-122.384424
3,Bayview,2014,35338,San Francisco,37.730416,-122.384424
4,Bayview,2015,42870,San Francisco,37.730416,-122.384424


In [7]:
home_df.head()

,Assessor Neighborhood,Assessed Land Value,Closed Roll Year
0,Alamo Square,356275.88,2007
1,Anza Vista,249759.91,2007
2,Balboa Terrace,308412.95,2007
3,Bayview,181096.93,2007
4,Bayview Heights,128157.54,2007


In [8]:
#rename columns to merge on same names
home_df = home_df.rename(columns={'Assessor Neighborhood': 'Neighborhood', 
                                  'Assessed Land Value': 'Home Price',
                                  'Closed Roll Year': 'Year'})

#rename home prices neighbourhoods for consistency with rent neighbourhoods
home_df["Neighborhood"] = home_df["Neighborhood"].replace({"Buena Vista Park": "Buena Vista",
                                                             "Bayview Heights": "Bayview",
                                                             "Croker Amazon": "Crocker Amazon",
                                                             "Forest Hill Extension": "Forest Hill",
                                                             "Lake --The Presidio": "Lake", 
                                                             "Lake Shore": "Lakeshore", 
                                                             "Jordan Park/Laurel Heights": "Laurel Heights",
                                                             "Mission Dolores": "Mission", 
                                                             "Sea Cliff": "Seacliff",
                                                             "Financial District North": "Financial District",
                                                             "Financial District South": "Financial District"})
# Group by the same neighbourhood name and use the means
home_gb = home_df.groupby(["Neighborhood", "Year"]).mean()
home_gb = home_gb.reset_index()

home_gb.head()

,Neighborhood,Year,Home Price
0,Alamo Square,2007,356275.88
1,Alamo Square,2008,384701.69
2,Alamo Square,2009,399153.26
3,Alamo Square,2010,411813.43
4,Alamo Square,2011,425384.56


In [9]:
#merge home price and rent dataframes
merged_df = pd.merge(home_gb,rent_df2, on = ["Neighborhood", "Year"])

merged_df.head()

,Neighborhood,Year,Home Price,Annual Rent,City,Lat,Lng
0,Bayview,2011,182276.165,30723,San Francisco,37.730416,-122.384424
1,Bayview,2012,187250.875,28821,San Francisco,37.730416,-122.384424
2,Bayview,2013,196027.270,30433,San Francisco,37.730416,-122.384424
3,Bayview,2014,201599.310,35338,San Francisco,37.730416,-122.384424
4,Bayview,2015,214597.840,42870,San Francisco,37.730416,-122.384424


In [10]:
#Grouping DataFrame based on "Neighborhood"
merged_gb = merged_df.groupby("Neighborhood").mean()

#Dropping column
merged_gb = merged_gb.drop(['Year'], axis=1)
merged_gb = merged_gb.reset_index()

merged_gb.head()

,Neighborhood,Home Price,Annual Rent,Lat,Lng
0,Bayview,201617.760000,35644.333333,37.730416,-122.384424
1,Bernal Heights,268491.455000,43599.666667,37.738950,-122.415201
2,Buena Vista,487987.843333,53112.333333,37.806504,-122.420778
3,Corona Heights,394108.123333,52034.000000,37.761812,-122.443185
4,Cow Hollow,614482.066667,63147.833333,37.798033,-122.439646


In [11]:
#save file
merged_df.to_csv('../Data/hp_and_rent.csv', sep=',', encoding='utf-8', index = False)
merged_gb.to_csv('../Data/hp_and_rent_grouped.csv', sep=',', encoding='utf-8', index = False)